In [ ]:
start_value = data['equity_curve'].iloc[0]
end_value = data['equity_curve'].iloc[-1]
expected_cumulative_return = (end_value - start_value) / start_value

data['cumulative_equity_returns'] = (1 + data['equity_returns']).cumprod()

plt.figure(figsize=(14, 7))
plt.plot(data['cumulative_equity_returns'], label='Cumulative Equity Returns')
plt.title('Cumulative Equity Returns Over Time')
plt.xlabel('Time')
plt.ylabel('Cumulative Returns')
plt.legend()
plt.show()

final_cumulative_return = data['cumulative_equity_returns'].iloc[-1] - 1
print(f'Calculated Cumulative Return from Equity Returns: {final_cumulative_return:.4f}')

print(data.head())
print(data.info())

In [ ]:
sell_quantiles = range(0, 5)
buy_quantiles = range(8, 12)  

def assign_trading_signals(data, quantile_summary, buy_quantiles, sell_quantiles):
    # Assign quantiles to signals
    bins = [-np.inf] + quantile_summary['signal']['max'].tolist() + [np.inf]
    
    data['signal_quantile'] = pd.cut(data['signal'], bins=bins, labels=False, include_lowest=True)
    # Initialize trading signals
    data['trade_signal'] = 0

    # Assign buy and sell signals
    data.loc[data['signal_quantile'].isin(buy_quantiles), 'trade_signal'] = 1
    data.loc[data['signal_quantile'].isin(sell_quantiles), 'trade_signal'] = -1

    data['strategy_returns'] = data['equity_returns'] * data['trade_signal']
    data['cumulative_strategy_returns'] = (1 + data['strategy_returns']).cumprod() - 1

    return data

test_data_with_signals = assign_trading_signals(test_data, group_summary, buy_quantiles, sell_quantiles)
print(test_data_with_signals[['signal', 'signal_quantile', 'trade_signal', 'equity_returns', 'strategy_returns', 'cumulative_strategy_returns']].head(20))


plt.figure(figsize=(12, 6))
plt.plot(test_data_with_signals['cumulative_strategy_returns'], label='Cumulative Strategy Returns')
plt.xlabel('Time')
plt.ylabel('Cumulative Returns')
plt.title('Cumulative Strategy Returns Over Time')
plt.legend()
plt.show()
